In [3]:
import os
import sys
import pandas as pd
import numpy as np
import re
from tabulate import tabulate  # For better table formatting

# --- Path Configuration ---
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

project_root = os.path.dirname(current_dir)
print(f"Project root: {project_root}")

if project_root not in sys.path:
    sys.path.append(project_root)

cleaned_data_path = os.path.join(project_root, 'data', 'cleaned')
os.makedirs(cleaned_data_path, exist_ok=True)
print(f"Cleaned data directory: {cleaned_data_path}")

# --- Import Custom Modules ---
from scripts.preprocessing.clean_reviews import clean_reviews
from scripts.preprocessing.normalize_dates import normalize_dates

# --- Load Raw Data ---
print("\n" + "="*50)
print("LOADING RAW DATA")
print("="*50)

raw_data_path = os.path.join(project_root, 'data', 'raw')
raw_dfs = []
bank_samples = {}

for filename in os.listdir(raw_data_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(raw_data_path, filename)
        try:
            df = pd.read_csv(file_path)
            raw_dfs.append(df)
            print(f"\nLoaded {len(df)} rows from {filename}")
            bank_name = df['bank_name'].iloc[0] if 'bank_name' in df.columns else filename.split('_')[0]
            bank_samples[bank_name] = df.head(3)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

if not raw_dfs:
    print("\nError: No CSV files loaded from raw data directory")
    exit()

# Display samples per bank
print("\n" + "="*50)
print("RAW DATA SAMPLES PER BANK")
print("="*50)
for bank, sample_df in bank_samples.items():
    print(f"\n{bank} sample:")
    print(tabulate(
        sample_df,
        headers='keys',
        tablefmt='psql',
        showindex=False,
        maxcolwidths=[40, 5, 12, 20, 15]
    ))

combined_df = pd.concat(raw_dfs, ignore_index=True)
print(f"\nTotal combined reviews: {len(combined_df)}")

# --- Apply Cleaning ---
print("\n" + "="*50)
print("APPLYING CLEANING OPERATIONS")
print("="*50)

column_mapping = {'review_text': 'review', 'bank_name': 'bank'}
combined_df = combined_df.rename(columns=column_mapping)

print("\nApplying text cleaning...")
cleaned_df = clean_reviews(combined_df)

print("\nApplying date normalization...")
normalized_df = normalize_dates(cleaned_df)

# Display cleaned data samples per bank
print("\n" + "="*50)
print("CLEANED DATA SAMPLES PER BANK")
print("="*50)

cleaned_samples = {}
for bank in normalized_df['bank'].unique():
    bank_df = normalized_df[normalized_df['bank'] == bank]
    cleaned_samples[bank] = bank_df.head(3)

    print(f"\n{bank} cleaned sample:")
    display_df = cleaned_samples[bank].copy()
    display_df['review'] = display_df['review'].apply(lambda x: (x[:70] + '...') if len(x) > 70 else x)
    print(tabulate(
        display_df,
        headers='keys',
        tablefmt='psql',
        showindex=False,
        maxcolwidths=[40, 5, 12, 20, 15]
    ))

# --- Data Quality Checks ---
print("\n" + "="*50)
print("DATA QUALITY CHECKS")
print("="*50)

print("\nMissing values after cleaning:")
print(normalized_df.isnull().sum())

print("\nDate validity check:")
print(f"Earliest date: {normalized_df['date'].min()}")
print(f"Latest date: {normalized_df['date'].max()}")

print("\nText length distribution:")
normalized_df['review_length'] = normalized_df['review'].apply(len)
print(normalized_df['review_length'].describe())

# --- Save Cleaned Data Per Bank ---
print("\n" + "="*50)
print("SAVING CLEANED DATA PER BANK")
print("="*50)

timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')

for bank_name in normalized_df['bank'].unique():
    bank_df = normalized_df[normalized_df['bank'] == bank_name]
    safe_bank_name = re.sub(r'\W+', '_', bank_name.strip().lower())
    bank_filename = f'cleaned_{safe_bank_name}_{timestamp}.csv'
    bank_output_path = os.path.join(cleaned_data_path, bank_filename)
    bank_df.to_csv(bank_output_path, index=False)
    print(f"✅ Saved {len(bank_df)} reviews for {bank_name} to {bank_output_path}")

# --- Final Exploration ---
print("\n" + "="*50)
print("FINAL DATA EXPLORATION")
print("="*50)

print("\nBank distribution:")
bank_counts = normalized_df['bank'].value_counts()
for bank, count in bank_counts.items():
    print(f"- {bank}: {count} reviews ({count/len(normalized_df):.1%})")

print("\nRating distribution:")
rating_counts = normalized_df['rating'].value_counts().sort_index()
print(rating_counts)

print("\nReview length by rating:")
print(normalized_df.groupby('rating')['review_length'].describe())

print("\n🎉 Cleaning process completed successfully!")


Current directory: /home/ablex/Development/AI/kifiya_week2/fintech-app-reviews/notebooks
Project root: /home/ablex/Development/AI/kifiya_week2/fintech-app-reviews
Cleaned data directory: /home/ablex/Development/AI/kifiya_week2/fintech-app-reviews/data/cleaned

LOADING RAW DATA

Loaded 450 rows from Dashen_Bank_reviews_20250609_230552.csv

Loaded 500 rows from Bank_of_Abyssinia_reviews_20250609_230608.csv

Loaded 500 rows from Commercial_Bank_of_Ethiopia_reviews_20250609_230534.csv

RAW DATA SAMPLES PER BANK

Dashen Bank sample:
+------------------------------------------+----------+------------+-------------+-------------+
| review_text                              |   rating | date       | bank_name   | source      |
|------------------------------------------+----------+------------+-------------+-------------|
| kalid                                    |        5 | 2025-06-08 | Dashen Bank | Google Play |
| I like this mobile banking app very      |        2 | 2025-06-07 | Dashen Ba

/home/ablex/Development/AI/kifiya_week2/fintech-app-reviews/scripts/preprocessing/clean_reviews.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["review"]= df["review"].apply(clean_text)
/home/ablex/Development/AI/kifiya_week2/fintech-app-reviews/scripts/preprocessing/normalize_dates.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
/home/ablex/Development/AI/kifiya_week2/fintech-app-reviews/scripts/preprocessing/normalize_dates.py: